In [29]:
import pandas as pd

In [30]:
business_df = pd.read_json('../dataset/business.json',orient='records', lines=True)


In [31]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174567 entries, 0 to 174566
Data columns (total 15 columns):
address         174567 non-null object
attributes      174567 non-null object
business_id     174567 non-null object
categories      174567 non-null object
city            174567 non-null object
hours           174567 non-null object
is_open         174567 non-null int64
latitude        174566 non-null float64
longitude       174566 non-null float64
name            174567 non-null object
neighborhood    174567 non-null object
postal_code     174567 non-null object
review_count    174567 non-null int64
stars           174567 non-null float64
state           174567 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 20.0+ MB


In [32]:
business_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


In [33]:
!python -m pip install missingno

In [34]:
import missingno as msno

In [35]:
msno.bar(business_df)

In [36]:
business_grouped_by_status = business_df[['is_open','stars']].groupby(by='is_open').sum().reset_index()
business_grouped_by_status.columns = ['open_status', 'count']
business_grouped_by_status.plot(kind='bar', x='open_status', y='count')


In [37]:
# eliminate restaurants that are closed
business_df = business_df[business_df['is_open'] == 1]

In [38]:
business_df.shape

(146702, 15)

In [39]:
most_business_info = business_df[['state', 'stars']].groupby(by='state').count().reset_index().sort_values(by='stars', ascending=False).head(5)
most_business_info.plot(kind='bar', x='state', y='stars')

In [40]:
top_states = most_business_info['state']
top_states_df = business_df[business_df['state'].isin(top_states)]
top_states_df.shape

(118278, 15)

In [41]:
state_cat_dict = dict()
def check_record(record) :
    state = record['state']
    categories = record.categories
    if state in state_cat_dict :
        state_obj = state_cat_dict[state]
        for cat in categories :
            if cat in state_obj :
                #cat exist
                state_obj[cat] = state_obj[cat] + 1
            else :
                # cat does not exist in dict
                state_obj[cat] = 1
        state_cat_dict[state] = state_obj
    else :
        # new state
        state_obj = dict()
        for cat in categories :
            state_obj[cat] = 1
        state_cat_dict[state] = state_obj
        
temp = top_states_df.apply(lambda x : check_record(x), axis=1)


In [42]:
category_dict = dict()
def check_record(record) :

    categories = record.categories
    for cat in categories :
        if cat in category_dict :
            #cat exist
            category_dict[cat] = category_dict[cat] + 1
        else :
            # cat does not exist in dict
            category_dict[cat] = 1
    
    
        
temp = top_states_df.apply(lambda x : check_record(x), axis=1)


In [43]:
required_category = ['Restaurants','Food','Nightlife','Bars','Fast Food','Sandwiches',
 'American (Traditional)','Pizza','Coffee & Tea','Burgers','Mexican','Breakfast & Brunch','Specialty Food',
 'Chinese','Italian','American (New)','Bakeries','Desserts','Chicken Wings','Salad','Beer','Wine & Spirits',
 'Pubs','Sushi Bars','Delis','Juice Bars & Smoothies','Steakhouses'
]
len(required_category)

27

In [44]:
state_cat_dict.keys()

dict_keys(['AZ', 'OH', 'NV', 'ON', 'NC'])

In [45]:
def check_if_restaurant(categories):
    for cat in categories :
        if cat in required_category :
            return True
    return False
    

In [46]:
top_states_df['restaurant'] = top_states_df.apply(lambda x : check_if_restaurant(x['categories']), axis = 1)


C:\Program Files\ADS\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [48]:
top_states_df = top_states_df[top_states_df['restaurant'] == True]

In [49]:
top_states_df.drop('restaurant', axis=1, inplace=True)

In [50]:
top_states_df.to_json('business_restaurant.json', orient='records', lines=True)

In [51]:
business_restaurant_df = pd.read_json('business_restaurant.json',orient='records', lines=True)
business_restaurant_df.shape

(39949, 15)

In [52]:
business_restaurant_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH
1,35 Main Street N,"{'BusinessParking': {'garage': False, 'street'...",xcgFnd-MwkZeO5G2HQ0gAQ,"[Bakeries, Bagels, Food]",Markham,{},1,43.875177,-79.260153,T & T Bakery and Cafe,Markham Village,L3P 1X3,38,4.0,ON
2,600 E 4th St,"{'GoodForMeal': {'dessert': False, 'latenight'...",fNMVV_ZX7CJSDWQGdOM8Nw,"[Restaurants, American (Traditional)]",Charlotte,"{'Friday': '7:00-15:00', 'Tuesday': '7:00-15:0...",1,35.221647,-80.839345,Showmars Government Center,Uptown,28202,7,3.5,NC
3,"9665 Bayview Avenue, Unit 1-4","{'RestaurantsTableService': True, 'GoodForMeal...",Gu-xs3NIQTj3Mj2xYoN2aw,"[French, Food, Bakeries, Restaurants]",Richmond Hill,"{'Monday': '11:30-22:00', 'Tuesday': '11:30-22...",1,43.867565,-79.412662,Maxim Bakery & Restaurant,,L4C 9V4,34,3.5,ON
4,85 Hanna Avenue,"{'BusinessParking': {'garage': False, 'street'...",lHYiCS-y8AFjUitv6MGpxg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '5:30-22:00', 'Tuesday': '5:30-22:0...",1,43.639863,-79.419533,Starbucks,Liberty Village,M6K 3S3,21,4.0,ON
